In [ ]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
chat_model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)

In [ ]:
human_message_prompt = HumanMessagePromptTemplate.from_template("Make up a funny name for a company that manufacture {product}")

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

In [ ]:
from langchain.chains import LLMChain

In [ ]:
chain_ = LLMChain(llm=chat_model, prompt=chat_prompt)

In [ ]:
result = chain_.invoke({"product":"Computers"})

In [ ]:
print (result["text"])

In [ ]:
#Second Example : Simple Sequential Chain

# 1. LLMChain => Outline for the blog taking {topic} from the user
# 2. LLMChain => Output of (1) as input and then write the complete blog
# 3. Simple Sequential Chain to sequence (1) and (2)

In [ ]:
#chain_one

template = "Give me a simple bullet point outline for a blog post on {topic}"
first_prompt = HumanMessagePromptTemplate.from_template(template=template)
first_chat_prompt = ChatPromptTemplate.from_messages([first_prompt])
chain_one = LLMChain(llm=chat_model, prompt=first_chat_prompt)

In [ ]:
#chain_two

template = "Write a blog post using the outline {outline}"
second_prompt = HumanMessagePromptTemplate.from_template(template=template)
second_chat_prompt = ChatPromptTemplate.from_messages([second_prompt])
chain_two = LLMChain(llm=chat_model, prompt=second_chat_prompt)

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
full_chain = SimpleSequentialChain(chains=[chain_one, chain_two],verbose=True)

In [ ]:
chain_two

In [ ]:
result = full_chain.invoke("Generative AI")

In [ ]:
print(result)

In [ ]:
print(result["output"])

In [ ]:
##Third Example : Sequential Chain
Ex: Performance Review Text -> Review Summary (Strengths & Weaknesses) -> Improvement Plan to address those weaknesses 
# 1. LLMChain => Outline for the blog taking {text} from the user
# 2. LLMChain => Output of (1) as input and then write the complete blog
# 3. LLMChain => Output of (3) as input and then write the complete blog
# 3. Sequential Chain to sequence (1),(2) and (3)

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
template1 = "Give a summary of employee's performance review : \n {review_text}" 
prompt1 = HumanMessagePromptTemplate.from_template(template=template1)
chat_prompt1 = ChatPromptTemplate.from_messages([prompt1])
chain1 = LLMChain(llm=chat_model, prompt=chat_prompt1, output_key="review_summary")

In [ ]:
template2 = "Identify key weaknesses of the employee from this review summary: \n{review_summary}" 
prompt2 = HumanMessagePromptTemplate.from_template(template=template2)
chat_prompt2 = ChatPromptTemplate.from_messages([prompt2])
chain2 = LLMChain(llm=chat_model, prompt=chat_prompt2, output_key="weaknesses")

In [ ]:
template3 = "Create a personalized improvement plan to address the weaknesses: \n{weaknesses}" 
prompt3 = HumanMessagePromptTemplate.from_template(template=template3)
chat_prompt3 = ChatPromptTemplate.from_messages([prompt3])
chain3 = LLMChain(llm=chat_model, prompt=chat_prompt3, output_key="improvement_plan")

In [ ]:
chain = SequentialChain(chains=[chain1,chain2,chain3],
                        input_variables=["review_text"],
                        output_variables=["review_summary","weaknesses","improvement_plan"],
                        verbose = True)

In [ ]:
review_text = """
Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexibility in handling changing project requirements and learning new technologies. This adaptability allows him to seamlessly transition between different projects and tasks, making him a valuable asset to the team.

Joe's problem-solving skills are exceptional. He approaches issues with a logical mindset and consistently finds effective solutions, often thinking outside the box. His ability to break down complex problems into manageable parts is key to his success in resolving issues efficiently.

Weaknesses:
While Joe possesses numerous strengths, there are a few areas where he could benefit from improvement. One such area is time management. Occasionally, Joe struggles with effectively managing his time, resulting in missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would greatly enhance his efficiency.

Another area for improvement is Joe's written communication skills. While he communicates well verbally, there have been instances where his written documentation lacked clarity, leading to confusion among team members. Focusing on enhancing his written communication abilities will help him effectively convey ideas and instructions.

Additionally, Joe tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging him to delegate tasks appropriately will not only alleviate his own workload but also foster a more balanced and productive team environment.
"""

In [ ]:
result = chain.invoke(review_text)

In [ ]:
print(result)

In [ ]:
print (result["review_text"])

In [ ]:
print(result["review_summary"])

In [ ]:
print (result["weaknesses"])

In [ ]:
print (result["improvement_plan"])